## Bechmarking using DeepEval

In [ ]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

### Step 1: Initialize Model Class (required to conduct bechmarking with DeepEval )

In [1]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer

# Initialize wandb
wandb.login()

# Load the latest model artifact from wandb
run = wandb.init(project="master_thesis_math_lm")
artifact = run.use_artifact("master_thesis_math_lm/gpt2-math/gpt2-math-model:v0", type="model")
artifact_dir = artifact.download()  # This will download the model to a local path

# Load the model from the downloaded path
model = AutoModelForCausalLM.from_pretrained(artifact_dir)
tokenizer = AutoTokenizer.from_pretrained(artifact_dir)

print("Model loaded successfully from Weights & Biases!")

/Users/konstantinreicheneder/Desktop/Master_Thesis/math-reasoning-in-language-models/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/konstantinreicheneder/.netrc
wandb: Currently logged in as: kreicheneder (kreicheneder-copenhagen-business-school). Use `wandb login --relogin` to forc

Model loaded successfully from Weights & Biases!


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class FineTunedGPT2:
    def __init__(self, model, tokenizer):
        """Initialize with preloaded model and tokenizer."""
        self.model = model
        self.tokenizer = tokenizer

        # Move model to the appropriate device
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def generate(self, prompt: str, max_length=100) -> str:
        """Generates text from the fine-tuned GPT-2 model."""
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(**inputs, max_new_tokens=max_length, do_sample=True)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Now, instead of reloading from wandb, pass your already loaded model & tokenizer
gpt2_model = FineTunedGPT2(model, tokenizer)

# Test text generation
#print(gpt2_model.generate("Tell me a math joke"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tell me a math joke or problem with physics? Let's think about it!

The math joke.

I don't know how they get on.

What do your friends say about it?

# 1.

### Eulal and he are in love at the moment, the latter loves to watch. He goes to the cinema to buy popcorn and he notices that he was asked if there is any movie with the same title or the same director.

#### Explanation:



### Benchmarking on GSM8K

In [4]:
from deepeval.benchmarks import GSM8K

# Define benchmark with n_problems and shots
benchmark = GSM8K(
    n_problems=10,
    n_shots=3,
    enable_cot=True
)

# Replace 'mistral_7b' with your own custom model
benchmark.evaluate(model=gpt2_model)
print(benchmark.overall_score)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processing 10 problems: 100%|██████████| 10/10 [00:25<00:00,  2.50s/it]

Overall GSM8K Accuracy: 0.0
0.0
